In [2]:
import torch
import ray
import gym
from IPython import display
import ray.rllib.agents.dqn as dqn
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
from ray.rllib.agents import dqn

config = dqn.DEFAULT_CONFIG.copy()
config['framework'] = 'torch'
trainer = dqn.DQNTrainer(env='Breakout-v0', config=config)
policy = trainer.get_policy()
model = policy.q_model
print(model)
for i in range(10):
    print(trainer.train()['episode_reward_mean'])

2020-10-11 16:42:58,521	WARNING util.py:37 -- Install gputil for GPU system monitoring.


VisionNetwork_as_DQNTorchModel(
  (_convs): Sequential(
    (0): SlimConv2d(
      (_model): Sequential(
        (0): ZeroPad2d(padding=(2, 2, 2, 2), value=0.0)
        (1): Conv2d(4, 16, kernel_size=[8, 8], stride=(4, 4))
        (2): ReLU()
      )
    )
    (1): SlimConv2d(
      (_model): Sequential(
        (0): ZeroPad2d(padding=(1, 2, 1, 2), value=0.0)
        (1): Conv2d(16, 32, kernel_size=[4, 4], stride=(2, 2))
        (2): ReLU()
      )
    )
    (2): SlimConv2d(
      (_model): Sequential(
        (0): Conv2d(32, 256, kernel_size=[11, 11], stride=(1, 1))
        (1): ReLU()
      )
    )
  )
  (_value_branch): SlimFC(
    (_model): Sequential(
      (0): Linear(in_features=256, out_features=1, bias=True)
    )
  )
  (advantage_module): Sequential(
    (dueling_A_0): SlimFC(
      (_model): Sequential(
        (0): Linear(in_features=256, out_features=256, bias=True)
        (1): ReLU()
      )
    )
    (A): SlimFC(
      (_model): Sequential(
        (0): Linear(in_featur

In [5]:
import logging

from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.utils.annotations import override
from ray.rllib.utils.framework import try_import_torch

torch, nn = try_import_torch()

logger = logging.getLogger(__name__)


class FullyConnectedNetwork(TorchModelV2, nn.Module):
    """Generic fully connected network."""

    def __init__(self, obs_space, action_space, num_outputs, model_config,
                 name):
        TorchModelV2.__init__(self, obs_space, action_space, num_outputs,
                              model_config, name)
        nn.Module.__init__(self)

        input_size = int(np.product(obs_space.shape))
        self._hidden_layers = nn.Sequential(
            torch.nn.Linear(input_size, 256), 
            torch.nn.ReLU(), 
            torch.nn.Linear(256, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, num_outputs))
        self._features = None

        

    @override(TorchModelV2)
    def forward(self, input_dict, state, seq_lens):
        obs = input_dict["obs_flat"].float()
        self._features = self._hidden_layers(obs)
        return self._features, state


In [6]:
import torch.nn as nn

import ray
from ray.rllib.agents import dqn
from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2

ray.shutdown()

ModelCatalog.register_custom_model("my_model", FullyConnectedNetwork)

ray.init()
trainer = dqn.DQNTrainer(env="Breakout-v0", config={
    "framework": "torch",
    "model": {
        "custom_model": "my_model",
        # Extra kwargs to be passed to your model's c'tor.
        "custom_model_config": {},
    },
})

2020-10-11 16:50:10,303	INFO resource_spec.py:231 -- Starting Ray with 3.56 GiB memory available for workers and up to 1.8 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-10-11 16:50:10,774	INFO services.py:1193 -- View the Ray dashboard at localhost:8265
2020-10-11 16:50:11,555	WARNING util.py:37 -- Install gputil for GPU system monitoring.


In [7]:

policy = trainer.get_policy()
model = policy.q_model
print(model)
for i in range(10):
    print(trainer.train()['episode_reward_mean'])

FullyConnectedNetwork_as_DQNTorchModel(
  (_hidden_layers): Sequential(
    (0): Linear(in_features=28224, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=256, bias=True)
  )
  (advantage_module): Sequential(
    (dueling_A_0): SlimFC(
      (_model): Sequential(
        (0): Linear(in_features=256, out_features=256, bias=True)
        (1): ReLU()
      )
    )
    (A): SlimFC(
      (_model): Sequential(
        (0): Linear(in_features=256, out_features=4, bias=True)
      )
    )
  )
  (value_module): Sequential(
    (dueling_V_0): SlimFC(
      (_model): Sequential(
        (0): Linear(in_features=256, out_features=256, bias=True)
        (1): ReLU()
      )
    )
    (V): SlimFC(
      (_model): Sequential(
        (0): Linear(in_features=256, out_features=1, bias=True)
      )
    )
  )
)
1.6666666666666667
1.375
1.5833333333333333
1.4705882352941178
1.523809523